<a href="https://www.kaggle.com/code/emanoelcarvalholopes/uci-lr-students-performance-prediction?scriptVersionId=214942497" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Apresentação

Este trabalho visa elaborar um modelo de predição da nota final dos estudantes utilizando regressão logística. O trabalho balisador é o artigo da Vanessa Faria de Souza que utiliza [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/320/student+performance). 

> “Na primeira etapa os dados foram coletados do repositório de dados público o UCIMachineLearning. Estes dados abordam o desempenho dos alunos no ensino secundário de duas escolas portuguesas.” (Faria De Souza, 2021, p. 773)

Além do artigo, será utilizado o livro do Géron, capítulo 2, onde os seguintes passos são apresentados:

1. Olhar para o quadro geral;
2. Obter os dados;
3. Descobrir e visualizar os dados para obter informações;
4. Preparar os dados para os algoritmos do Aprendizado de Máquina;
5. Selecionar e treinar um modelo;
6. Ajustar o seu modelo;
7. Apresentar sua solução;

Os dados serão obtidos por meio da instalação do repositório da UCI, por meio do comando: 

!pip install ucimlrepo


# 0 - Importando as bibliotecas

In [1]:
# importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 2. Obter os dados

In [2]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
students = fetch_ucirepo(id=320) 

In [4]:
students.data.features.shape

(649, 30)

In [5]:
students.data.targets.shape

(649, 3)

In [6]:
# Junção das duas bases de dados 
# students.data.features + students_data.targets 

students_full = pd.merge(students.data.features,students.data.targets, left_index = True, right_index = True, how = 'inner')

Agora, temos o dataset unificado. 

In [7]:
students_full.shape

(649, 33)

# 3. Preparação dos dados 

## Ordinal Encoding

Atributos categóricos transformados em números ordinais. 

In [8]:
# Separar a classe alvo das variáveis independentes

y = students_full['G3']

X = students_full.iloc[:,:-1] # atributos do início até o penúltimo


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# Colunas com atributos categóricos
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_test[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

Categorical columns that will be dropped from the dataset: []


In [10]:
# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_test = X_test.drop(bad_label_cols, axis=1)

# Apply ordinal encoder 
enc =  OrdinalEncoder()
label_X_train[good_label_cols] = enc.fit_transform(X_train[good_label_cols])
label_X_test[good_label_cols] = enc.transform(X_test[good_label_cols])


In [11]:
# function for comparing different approaches
def score_dataset(X_train, X_test, y_train, y_test):
    model = LogisticRegression(random_state=42, max_iter=200)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

In [12]:
print("MAE para Ordinal Encoding:") 
print(score_dataset(label_X_train, label_X_test, y_train, y_test))

MAE para Ordinal Encoding:
1.1226993865030674


## One Hot Enconding

Nesta abordagem, os atributos categóricos serão tratados com a técnica do one hot enconding

In [ ]:
one_hot_encoded_training_predictors = pd.get_dummies(students_full.iloc[:,:-1]) # todas as colunas, exceto a última: nota final (G3)

In [ ]:
# Separar a classe alvo das variáveis independentes

y = students_full['G3']

# X = students_full.iloc[:,:-1] # atributos do início até o penúltimo



In [ ]:
# Transformação de atributos categóricos independentes

X = one_hot_encoded_training_predictors

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Utilizando o modelo de Regressão Logística

log_reg = LogisticRegression(random_state=42, max_iter=200)
log_reg.fit(X_train, y_train)

In [ ]:
# Predições

y_pred = log_reg.predict(X_test)
y_pred_prob = log_reg.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Acurácia: {accuracy:.2f}")

### Observação: este modelo precisa ser aprimorado.

In [ ]:
# Utilizando o modelo de Árvore de Decisão

In [ ]:
students_num = students_full.drop('G3', axis=1)